### In this notebook, we analyze whether the use of our method helps speed up feature selection as a preprocessing, pruning step. Naturally, it only makes sense to do so if the "pre-pruned" results are compatible in quality with what we would get with the pruners.

### Let's assume for now that the dataset in question involves using the number of taxi trips to predict the number  of accidents in traffic.

In [114]:
import pandas as pd
import json
import os
import warnings; warnings.simplefilter('ignore')
from sklearn.impute import SimpleImputer
import numpy as np


In [45]:
taxi_trips = pd.read_csv('nyc_indicators/taxi_count.csv', sep=SEPARATOR)
taxi_trips = taxi_trips.rename(columns={'count': 'taxi_count'})
traffic_accidents = pd.read_csv('nyc_indicators/crash_count.csv', sep=SEPARATOR)
traffic_accidents = traffic_accidents.rename(columns={'count': 'crash_count'})
taxi_crash = pd.merge(taxi_trips, traffic_accidents, on='time', how='inner')

In [571]:
def join_datasets(base_dataset, dataset_directory, key, mean_data_imputation=True, rename_numerical=True, separator='|'):
    '''
    Given (1) a base dataset, (2) a directory with datasets that only have two 
    columns (one key and one numerical attribute), and (3) a key that is present 
    in all of them and helps for joining purposes, this function generates a big
    table composed of all joined datasets.
    '''
    
    augmented_dataset = base_dataset
    dataset_names = [f for f in os.listdir(dataset_directory) if '.csv' in f]
    for name in dataset_names:
        try:
            ### Step 1: read the dataset in the directory
            dataset = pd.read_csv(os.path.join(dataset_directory, name), 
                                  sep=separator)
            
            ### Step 2 (optional):  rename the numerical column in the dataset
            if rename_numerical:
                numerical_column = [i for i in dataset.columns if i != key][0]
                dataset = dataset.rename(columns={numerical_column: name.split('.')[0]})
    
            ### Step 3: augment the table
            augmented_dataset = pd.merge(augmented_dataset, 
                                         dataset,
                                         how='left',
                                         on=key)
        except pd.errors.EmptyDataError:
            continue
    
    augmented_dataset = augmented_dataset.set_index(key)

    if mean_data_imputation:
        fill_NaN = SimpleImputer(missing_values=np.nan, strategy='mean')
        new_data = pd.DataFrame(fill_NaN.fit_transform(augmented_dataset))
        new_data.index = augmented_dataset.index
        new_data.columns = augmented_dataset.columns
        return new_data
    
    return augmented_dataset

In [95]:
augmented_dataset = join_datasets(taxi_crash, 'nyc_indicators/', 'time')

In [96]:
augmented_dataset.head()

,taxi_count,crash_count,311_category_Agency_Issues_added_zeros,taxispeed_speed_avg,311_category_SCRIE_added_zeros,cyclist_killed_sum,311_category_electric_added_zeros,311_category_Illegal_parking_added_zeros,311_category_Vacant_Lot_added_zeros,311_category_consumer_complaint_added_zeros,...,311_category_Noise_added_zeros,311_category_Literature_request_added_zeros,311_category_taxi_added_zeros,311_category_collection_added_zeros,311_category_homeless_person_assistance_added_zeros,311_category_Traffic_added_zeros,311_category_Damaged_Tree_added_zeros,motorist_killed_sum,311_category_Enforcement_added_zeros,311_category_graffiti_added_zeros
time,,,,,,,,,,,,,,,,,,,,,
2012-07-01,354746.0,443.0,0.0,17.902601,1.0,0.0,89.0,72.0,5.0,25.0,...,934.0,9.0,25.0,70.0,26.341051,61.0,32.0,0.0,13.0,6.0
2012-07-02,412337.0,475.0,0.0,16.507096,74.0,0.0,177.0,93.0,15.0,72.0,...,453.0,112.0,40.0,24.0,26.341051,88.0,80.0,1.0,123.0,169.0
2012-07-03,495375.0,577.0,0.0,15.954152,63.0,0.0,140.0,102.0,14.0,57.0,...,487.0,85.0,28.0,48.0,26.341051,121.0,73.0,1.0,58.0,38.0
2012-07-04,345717.0,353.0,0.0,17.808810,4.0,0.0,61.0,51.0,6.0,18.0,...,882.0,10.0,31.0,19.0,26.341051,70.0,25.0,0.0,8.0,5.0
2012-07-05,417036.0,517.0,0.0,16.603352,52.0,0.0,201.0,108.0,10.0,66.0,...,617.0,84.0,46.0,2.0,26.341051,100.0,60.0,0.0,71.0,96.0


### Now let's see which of these features would be classified by our approach as bad for augmentation. To this end, let's build our model over openml-training instances with $\theta = 1$, svm-rbf, and one candidate per query in the training examples. NOTE THAT THESE TRAINING INSTANCES REFER TO REGRESSION PROBLEMS.

### For now we are going to use the following class policy: if the gain in R2-score is predicted as "above zero", we consider that the feature should not be pruned. Otherwise it should.

In [169]:
from sklearn.svm import SVC
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

FEATURES = ['query_num_of_columns', 'query_num_of_rows', 'query_row_column_ratio',
            'query_max_skewness', 'query_max_kurtosis', 'query_max_unique', 
            'candidate_num_rows', 'candidate_max_skewness', 'candidate_max_kurtosis',
            'candidate_max_unique', 'query_target_max_pearson', 
            'query_target_max_spearman', 'query_target_max_covariance', 
            'query_target_max_mutual_info', 'candidate_target_max_pearson', 
            'candidate_target_max_spearman', 'candidate_target_max_covariance', 
            'candidate_target_max_mutual_info']
THETA = 1

def train_rbf_svm(features, classes):
    '''
    Builds a model using features to predict associated classes,
    '''

    feature_scaler = MinMaxScaler().fit(features)
    features_train = feature_scaler.transform(features)
    clf = SVC(max_iter=1000, gamma='auto')
    clf.fit(features_train, classes)

    return feature_scaler, clf

In [170]:
openml_training = pd.read_csv('../classification/training-simplified-data-generation.csv')
openml_training['class_pos_neg'] = ['gain' if row['gain_in_r2_score'] > 0 else 'loss' 
                                    for index, row in openml_training.iterrows()]
openml_training_high_containment = openml_training.loc[openml_training['containment_fraction'] >= THETA]
openml_training_high_containment.shape

(7566, 36)

In [171]:
feature_scaler, model = train_rbf_svm(openml_training_high_containment[FEATURES], 
                                      openml_training_high_containment['class_pos_neg'])

### Now, for each single-feature dataset in nyc_indicators/, we will join it with the base table, generate all features and classify it.

In [103]:
# The next two lines are important for importing files that are in the parent directory, 
# necessary to generate the features
import sys
sys.path.append('../')
from feature_factory import *

In [402]:
def compute_features(query_dataset, 
                     candidate_dataset, 
                     key, 
                     target_name, 
                     augmented_dataset=pd.DataFrame([]),
                     mean_data_imputation=True):
    '''
    This function generates all the features required to determine, through classification, 
    whether an augmentation with the candidate_dataset (which is single-feature) is likely to 
    hamper the model (or simply bring no gain)
    '''
    
    # Step 1: individual query features
    feature_factory_query = FeatureFactory(query_dataset.drop([target_name], axis=1))
    query_dataset_individual_features = feature_factory_query.get_individual_features(func=max_in_modulus)
    ## In order, the returned features are number_of_columns, number_of_rows, row_to_column_ratio,
    ## max_mean, max_outlier_percentage, max_skewness, max_kurtosis, max_number_of_unique_values.
    ## For now, we're only using number_of_columns, number_of_rows, row_to_column_ratio, 
    ## max_skewness, max_kurtosis, max_number_of_unique_values, so we remove the unnecessary elements 
    ## in the lines below
    query_dataset_individual_features = [query_dataset_individual_features[index] for index in [0, 1, 2, 5, 6, 7]]
    
    # Step 2: individual candidate features
    feature_factory_candidate = FeatureFactory(candidate_dataset)
    candidate_dataset_individual_features = feature_factory_candidate.get_individual_features(func=max_in_modulus)
    ## For now, we're only using number_of_rows, max_skewness, max_kurtosis, max_number_of_unique_values, 
    ## so we remove the unnecessary elements in the lines below 
    candidate_dataset_individual_features = [candidate_dataset_individual_features[index] for index in [1, 5, 6, 7]]
    
    # Step 3: join the datasets and compute pairwise features
    if augmented_dataset.empty:
        augmented_dataset = pd.merge(query_dataset, 
                                     candidate_dataset,
                                     how='left',
                                     on=key)
    #augmented_dataset = augmented_dataset.set_index(key)
    if mean_data_imputation:
        fill_NaN = SimpleImputer(missing_values=np.nan, strategy='mean')
        new_dataset = pd.DataFrame(fill_NaN.fit_transform(augmented_dataset))
        new_dataset.columns = augmented_dataset.columns
        new_dataset.index = augmented_dataset.index
        augmented_dataset = new_dataset
    
    # Step 3.1: get query-target features 
    ## The features are, in order: max_query_target_pearson, max_query_target_spearman, 
    ## max_query_target_covariance, max_query_target_mutual_info
    feature_factory_full_query = FeatureFactory(query_dataset)
    query_features_target = feature_factory_full_query.get_pairwise_features_with_target(target_name,
                                                                                         func=max_in_modulus)
    # Step 3.2: get candidate-target features
    ## The features are, in order: max_query_candidate_pearson, max_query_candidate_spearman, 
    ## max_query_candidate_covariance, max_query_candidate_mutual_info
    column_names = candidate_dataset.columns.tolist() + [target_name]
    feature_factory_candidate_target = FeatureFactory(augmented_dataset[column_names])
    candidate_features_target = feature_factory_candidate_target.get_pairwise_features_with_target(target_name,
                                                                                                   func=max_in_modulus)
    
    # Step 4: get query-candidate feature "containment ratio". We may not use it in models, but it's 
    ## important to have this value in order to filter candidates in baselines, for example.
    query_key_values = query_dataset.index.values
    candidate_key_values = candidate_dataset.index.values
    intersection_size = len(set(query_key_values) & set(candidate_key_values))
    containment_ratio = [intersection_size/len(query_key_values)]

    return np.array(query_dataset_individual_features + 
                    candidate_dataset_individual_features + 
                    query_features_target + 
                    candidate_features_target + 
                    containment_ratio)

candidate_dataset = pd.read_csv('nyc_indicators/citibike_count.csv', sep=SEPARATOR)
candidate_dataset = candidate_dataset.rename(columns={'count':'citibike_count'})
features = compute_features(taxi_crash.set_index('time'), 
                            candidate_dataset.set_index('time'), 
                            'time',
                            'crash_count')

In [259]:
print(features)

[1.00000000e+00 1.46200000e+03 1.46200000e+03 3.86883428e-01
 1.47702153e+00 1.45600000e+03 1.63300000e+03 3.02000052e-01
 5.28414244e-01 1.60300000e+03 1.60072219e-01 1.24021640e-01
 9.50707731e+05 8.68223838e-01 2.62378575e-01 2.87505489e-01
 2.27171493e+05 7.52732960e-01 7.47606019e-01]


### Let's quickly sanity-check these features.

In [152]:
print('query number of rows', taxi_crash.shape[0], 
      'query skewness', taxi_crash['taxi_count'].skew(), 
      'query kurtosis', taxi_crash['taxi_count'].kurtosis(),
      'candidate number of unique values', len(set(taxi_crash['taxi_count'])))

print('candidate number of rows', candidate_dataset.shape[0],
      'candidate skewness', candidate_dataset['citibike_count'].skew(), 
      'candidate kurtosis', candidate_dataset['citibike_count'].kurtosis(),
      'candidate number of unique values', len(set(candidate_dataset['citibike_count'])))

augmented_dataset = pd.merge(taxi_crash,
                             candidate_dataset,
                             how='left',
                             on='time').set_index('time')
fill_NaN = SimpleImputer(missing_values=np.nan, strategy='mean')
new_dataset = pd.DataFrame(fill_NaN.fit_transform(augmented_dataset))
new_dataset.columns = augmented_dataset.columns
new_dataset.index = augmented_dataset.index
augmented_dataset = new_dataset

from scipy.stats import pearsonr, spearmanr
from scipy import cov
from sklearn.metrics.cluster import normalized_mutual_info_score

print('query target pearson', pearsonr(augmented_dataset['taxi_count'], augmented_dataset['crash_count']),
      'query target spearman', spearmanr(augmented_dataset['taxi_count'], augmented_dataset['crash_count']),
      'query target covariance', cov(augmented_dataset['taxi_count'], augmented_dataset['crash_count'])[0, 1],
      'query target mutual info', normalized_mutual_info_score(augmented_dataset['taxi_count'], augmented_dataset['crash_count']))

print('candidate target pearson', pearsonr(augmented_dataset['citibike_count'], augmented_dataset['crash_count']),
      'candidate target spearman', spearmanr(augmented_dataset['citibike_count'], augmented_dataset['crash_count']),
      'candidate target covariance', cov(augmented_dataset['citibike_count'], augmented_dataset['crash_count'])[0, 1],
      'candidate target mutual info', normalized_mutual_info_score(augmented_dataset['citibike_count'], augmented_dataset['crash_count']))


query number of rows 1462 query skewness -0.38688342830440225 query kurtosis 1.4770215280107495 candidate number of unique values 1456
candidate number of rows 1633 candidate skewness 0.30200005209988534 candidate kurtosis -0.5284142437311292 candidate number of unique values 1603
query target pearson (0.160072218826492, 7.506211089340906e-10) query target spearman SpearmanrResult(correlation=0.12402163995232865, pvalue=1.970536577824118e-06) query target covariance 950707.7308413646 query target mutual info 0.8682238377526429
candidate target pearson (0.2623785747241163, 1.9218632466061835e-24) candidate target spearman SpearmanrResult(correlation=0.2875054890817096, pvalue=3.2000264303199555e-29) candidate target covariance 227171.49311560777 candidate target mutual info 0.7527329601416918


### Ok! So let's see what label is predicted once we present these features to our model

In [174]:
def normalize_features(features, scaler=None):
    '''
    This function normalizes features using sklearn's StandardScaler
    '''
    if not scaler:
        scaler = MinMaxScaler().fit(features)
    return scaler.transform(features)

print(model.predict(normalize_features([features], feature_scaler)))

['loss']


### Now let's create predictions for all candidates

In [188]:
candidate_names = [f for f in os.listdir('nyc_indicators/') if '.csv' in f]
feature_vectors = []
for name in candidate_names:
        candidate_dataset = pd.read_csv(os.path.join('nyc_indicators/', name),
                                        sep=SEPARATOR)
        numerical_column = [i for i in candidate_dataset.columns if i != 'time'][0]
        candidate_dataset = candidate_dataset.rename(columns={numerical_column: name.split('.')[0]})
        features = compute_features(taxi_crash.set_index('time'), 
                                    candidate_dataset.set_index('time'), 
                                    'time',
                                    'crash_count')
        feature_vectors.append(features)

In [189]:
predictions = model.predict(normalize_features(np.array(feature_vectors)))

### Additionally, let's see what gain is brought by each feature individually to an initial prediction model

In [222]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

def  compute_adjusted_r2_score(number_of_regressors, r2_score, number_of_samples):
    '''
    This function adjusts the value of an r2 score based on its numbers 
    of regressors and samples.
    '''
    try:
        num = (1.0 - r2_score) * (number_of_samples - 1.0)
        den = number_of_samples - number_of_regressors - 1.0
        return 1.0 - (num/den)  
    except ZeroDivisionError:
        print('**** ERROR', number_of_samples, number_of_regressors)
        return r2_score
    
def compute_model_performance_improvement(query_dataset, 
                                          candidate_dataset, 
                                          target_name, 
                                          key, 
                                          mean_data_imputation=True, 
                                          adjusted_r2_score=False, 
                                          model=None):
    '''
    This function computes the change in (adjusted) R2 score obtained when we try to predict 'target_name' with an 
    augmented dataset (query_dataset + candidate_dataset)
    '''
    
    # To make sure that we compare apples to apples, let's perform the augmentation and any 
    # necessary missing data imputation first
    augmented_dataset = pd.merge(query_dataset, 
                                 candidate_dataset,
                                 how='left',
                                 on=key)
    
    if mean_data_imputation:
        fill_NaN = SimpleImputer(missing_values=np.nan, strategy='mean')
        new_dataset = pd.DataFrame(fill_NaN.fit_transform(augmented_dataset))
        new_dataset.columns = augmented_dataset.columns
        new_dataset.index = augmented_dataset.index
        augmented_dataset = new_dataset
    
    # Now let's split the data
    X_train, X_test, y_train, y_test = train_test_split(augmented_dataset.drop([target_name], axis=1), 
                                                        augmented_dataset[target_name], 
                                                        test_size=0.33, 
                                                        random_state=42)
    
    # Computing the initial and final r-squared scores
    if not model:
        model = RandomForestRegressor(n_estimators=100, random_state=42)
    
    model.fit(X_train[query_dataset.drop([target_name], axis=1).columns], y_train.ravel())
    y_pred_initial = model.predict(X_test[query_dataset.drop([target_name], axis=1).columns])
    r2_score_initial = r2_score(y_test, y_pred_initial)
    if adjusted_r2_score:
        r2_score_initial = compute_adjusted_r2_score(len(query_dataset.drop([target_name], axis=1).columns),
                                                     r2_score_initial, 
                                                     len(y_test))
        
    model.fit(X_train[augmented_dataset.drop([target_name], axis=1).columns], y_train.ravel())
    y_pred_final = model.predict(X_test[augmented_dataset.drop([target_name], axis=1).columns])
    r2_score_final = r2_score(y_test, y_pred_final)
    if adjusted_r2_score:
        r2_score_final = compute_adjusted_r2_score(len(augmented_dataset.drop([target_name], axis=1).columns),
                                                   r2_score_final,
                                                   len(y_test))
    
    performance_difference = (r2_score_final - r2_score_initial)/np.fabs(r2_score_initial)
    
    return r2_score_initial, r2_score_final, performance_difference

In [191]:
true_labels = []
gains_in_r2_score = []

for name in candidate_names:
        candidate_dataset = pd.read_csv(os.path.join('nyc_indicators/', name),
                                        sep=SEPARATOR)
        numerical_column = [i for i in candidate_dataset.columns if i != 'time'][0]
        candidate_dataset = candidate_dataset.rename(columns={numerical_column: name.split('.')[0]})
        initial, final, improvement = compute_model_performance_improvement(taxi_crash.set_index('time'), 
                                                                            candidate_dataset.set_index('time'), 
                                                                            'crash_count', 
                                                                            'time')
        gains_in_r2_score.append(improvement)
        if improvement > 0:
            true_labels.append('gain')
        else:
            true_labels.append('loss')
        
        
candidates_info = {}
for name, pred, feat_vector, gain, true_label in zip(candidate_names, predictions, feature_vectors, gains_in_r2_score, true_labels):
    candidates_info[name] = (feat_vector, gain, pred, true_label)


### Let's check how well our model works for this example...

In [193]:
print(classification_report(true_labels, predictions))

              precision    recall  f1-score   support

        gain       0.99      0.86      0.92        80
        loss       0.00      0.00      0.00         1

    accuracy                           0.85        81
   macro avg       0.49      0.43      0.46        81
weighted avg       0.97      0.85      0.91        81



### This example is too easy in the sense that most candidates do lead to good augmentations. I need to create an example where most candidates do not help. 

### Let me first see which candidates brought the highest gain.

In [197]:
tmp = []
for name in candidate_names:
    tmp.append((name, candidates_info[name][1]))
print(sorted(tmp, key=lambda x: x[1]))

[('cyclist_killed_sum.csv', -0.013631028026993834), ('311_category_Boilers_added_zeros.csv', 0.07997961729100868), ('motorist_killed_sum.csv', 0.08119269554132075), ('persons_killed_sum.csv', 0.14147012684452737), ('311_category_Snow_added_zeros.csv', 0.1472460737106083), ('weather_snow_mean.csv', 0.16316251153613825), ('pedestrians_killed_sum.csv', 0.195407026426819), ('311_category_Animal_in_a_Park_added_zeros.csv', 0.21614843896030195), ('311_category_Violation_of_Park_Rules_added_zeros.csv', 0.24732513214012192), ('311_category_Smoking_added_zeros.csv', 0.3475493857864856), ('311_category_Vacant_Lot_added_zeros.csv', 0.4318863599771732), ('311_category_Drinking_added_zeros.csv', 0.43968255649418525), ('311_category_Lead_added_zeros.csv', 0.4920252978669224), ('311_category_Street_Sign_added_zeros.csv', 0.5369216015199827), ('cyclist_injured_sum.csv', 0.5549550374398279), ('311_category_Heating_added_zeros.csv', 0.5638690035157186), ('311_category_collection_added_zeros.csv', 0.5661

### A few of them seem to bring gains by accident, but let's see what happens if we start with a bigger table including taxispeed_speed_avg.csv and 311_category_Street_light_condition_added_zeros.csv.

In [201]:
taxispeed = pd.read_csv('nyc_indicators/taxispeed_speed_avg.csv', sep=SEPARATOR)
taxispeed = taxispeed.rename(columns={'mean': 'taxispeed'})
streetlight_complaints = pd.read_csv('nyc_indicators/311_category_Street_light_condition_added_zeros.csv', sep=SEPARATOR)
streetlight_complaints = streetlight_complaints.rename(columns={'count':'streetlight_complaints'})
taxi_crash_taxispeed = pd.merge(taxi_crash, taxispeed, on='time', how='inner')
taxi_crash_taxispeed_streetlight = pd.merge(taxi_crash_taxispeed, streetlight_complaints, on='time', how='inner')

taxi_crash_taxispeed_streetlight.head()

,time,taxi_count,crash_count,taxispeed,streetlight_complaints
0,2012-07-01,354746,443,17.902601,37
1,2012-07-02,412337,475,16.507096,410
2,2012-07-03,495375,577,15.954152,300
3,2012-07-04,345717,353,17.808810,31
4,2012-07-05,417036,517,16.603352,290


In [204]:
new_candidate_names = list(set(candidate_names) - set(['nyc_indicators/taxispeed_speed_avg.csv', 
                                         'nyc_indicators/311_category_Street_light_condition_added_zeros.csv']))

In [207]:
names_improvements = []
for name in new_candidate_names:
        candidate_dataset = pd.read_csv(os.path.join('nyc_indicators/', name),
                                        sep=SEPARATOR)
        numerical_column = [i for i in candidate_dataset.columns if i != 'time'][0]
        candidate_dataset = candidate_dataset.rename(columns={numerical_column: name.split('.')[0]})
        initial, final, improvement = compute_model_performance_improvement(taxi_crash_taxispeed_streetlight.set_index('time'), 
                                                                            candidate_dataset.set_index('time'), 
                                                                            'crash_count', 
                                                                            'time')
        gains_in_r2_score.append(improvement)
        names_improvements.append((name, improvement))
        if improvement > 0:
            true_labels.append('gain')
        else:
            true_labels.append('loss')
print(sorted(names_improvements, key=lambda x: x[1]))

[('turnstile_count.csv', -0.04203859621234797), ('311_category_Fire_Safety_Director_-_F58_added_zeros.csv', -0.038392664194468615), ('311_category_Smoking_added_zeros.csv', -0.032251622626679396), ('311_category_Vending_added_zeros.csv', -0.026410682357405905), ('311_category_Elevator_added_zeros.csv', -0.021587380873384954), ('311_category_Food_Poisoning_added_zeros.csv', -0.020836944765305725), ('311_category_Housing_added_zeros.csv', -0.017828113175924074), ('311_category_Snow_added_zeros.csv', -0.010012004624826565), ('311_category_graffiti_added_zeros.csv', -0.009935261814290716), ('311_category_Housing_Options_added_zeros.csv', -0.007073414864474615), ('311_category_door_window_added_zeros.csv', -0.001869179999049201), ('311_category_Street_light_condition_added_zeros.csv', 0.0004541862035178613), ('motorist_killed_sum.csv', 0.0009208130232868357), ('cyclist_killed_sum.csv', 0.0025005275485507307), ('311_category_building_added_zeros.csv', 0.006593334845260233), ('weather_windspe

### Now there are more candidates that are bringing no gains, but a bunch of them still do. Let's add a few more features to the initial dataset.

In [208]:
persons_injured = pd.read_csv('nyc_indicators/persons_injured_sum.csv', sep=SEPARATOR)
persons_injured = persons_injured.rename(columns={'sum': 'persons_injured'})
motorist_injured = pd.read_csv('nyc_indicators/motorist_injured_sum.csv', sep=SEPARATOR)
motorist_injured = motorist_injured.rename(columns={'sum':'motorist_injured'})
pedestrians_injured = pd.read_csv('nyc_indicators/pedestrians_injured_sum.csv', sep=SEPARATOR)
pedestrians_injured = pedestrians_injured.rename(columns={'sum': 'pedestrians_injured'})

crash_many_predictors = pd.merge(taxi_crash_taxispeed_streetlight, persons_injured, on='time', how='inner')
crash_many_predictors = pd.merge(crash_many_predictors, motorist_injured, on='time', how='inner')
crash_many_predictors = pd.merge(crash_many_predictors, pedestrians_injured, on='time', how='inner')

In [209]:
crash_many_predictors.head()

,time,taxi_count,crash_count,taxispeed,streetlight_complaints,persons_injured,motorist_injured,pedestrians_injured
0,2012-07-01,354746,443,17.902601,37,143,104,30
1,2012-07-02,412337,475,16.507096,410,138,91,24
2,2012-07-03,495375,577,15.954152,300,187,139,34
3,2012-07-04,345717,353,17.808810,31,125,100,15
4,2012-07-05,417036,517,16.603352,290,124,86,30


In [476]:
names_improvements = []
candidate_names = [f for f in os.listdir('nyc_indicators/') if '.csv' in f]

for name in candidate_names:
        candidate_dataset = pd.read_csv(os.path.join('nyc_indicators/', name),
                                        sep=SEPARATOR)
        numerical_column = [i for i in candidate_dataset.columns if i != 'time'][0]
        candidate_dataset = candidate_dataset.rename(columns={numerical_column: name.split('.')[0]})
        initial, final, improvement = compute_model_performance_improvement(crash_many_predictors.set_index('time'), 
                                                                            candidate_dataset.set_index('time'), 
                                                                            'crash_count', 
                                                                            'time')
        gains_in_r2_score.append(improvement)
        names_improvements.append((name, improvement))
        if improvement > 0:
            true_labels.append('gain')
        else:
            true_labels.append('loss')
print(sorted(names_improvements, key=lambda x: x[1]))

[('311_category_SCRIE_added_zeros.csv', -0.014791401791830274), ('311_category_Elevator_added_zeros.csv', -0.011451887625023294), ('311_category_graffiti_added_zeros.csv', -0.010378240905085438), ('311_category_Drinking_added_zeros.csv', -0.009650693505290275), ('311_category_Housing_Options_added_zeros.csv', -0.009210933861856407), ('311_category_Non-Emergency_Police_Matter_added_zeros.csv', -0.009008898241276646), ('311_category_Agency_Issues_added_zeros.csv', -0.006840305893750853), ('311_category_Enforcement_added_zeros.csv', -0.006730938137458545), ('311_category_DOH_New_License_Application_Request_added_zeros.csv', -0.006630506830875576), ('weather_windspeed_mean.csv', -0.00590437224095922), ('311_category_taxi_added_zeros.csv', -0.005281794934152415), ('311_category_School_Maintenance_added_zeros.csv', -0.004932540289080163), ('cyclist_killed_sum.csv', -0.0041670723365609405), ('311_category_Smoking_added_zeros.csv', -0.003993332663934144), ('311_category_Boilers_added_zeros.csv

### Now the gains are considerably lower and much closer to zero, but many are still slightly above zero. What would happen if we used 'gains in adjusted R2-score' instead of 'gains in R2-score'?

In [477]:
names_improvements = []
for name in candidate_names:
        candidate_dataset = pd.read_csv(os.path.join('nyc_indicators/', name),
                                        sep=SEPARATOR)
        numerical_column = [i for i in candidate_dataset.columns if i != 'time'][0]
        candidate_dataset = candidate_dataset.rename(columns={numerical_column: name.split('.')[0]})
        initial, final, improvement = compute_model_performance_improvement(crash_many_predictors.set_index('time'), 
                                                                            candidate_dataset.set_index('time'), 
                                                                            'crash_count', 
                                                                            'time', 
                                                                            adjusted_r2_score=True)
        gains_in_r2_score.append(improvement)
        names_improvements.append((name, improvement))
        if improvement > 0:
            true_labels.append('gain')
        else:
            true_labels.append('loss')
print(sorted(names_improvements, key=lambda x: x[1]))

[('311_category_SCRIE_added_zeros.csv', -0.0161776045624857), ('311_category_Elevator_added_zeros.csv', -0.012767376762512255), ('311_category_graffiti_added_zeros.csv', -0.011670995759528338), ('311_category_Drinking_added_zeros.csv', -0.01092804266975232), ('311_category_Housing_Options_added_zeros.csv', -0.01047897119215519), ('311_category_Non-Emergency_Police_Matter_added_zeros.csv', -0.010272657502357395), ('311_category_Agency_Issues_added_zeros.csv', -0.0080581455880649), ('311_category_Enforcement_added_zeros.csv', -0.007946461988505446), ('311_category_DOH_New_License_Application_Request_added_zeros.csv', -0.00784390406642779), ('weather_windspeed_mean.csv', -0.0071023937025346505), ('311_category_taxi_added_zeros.csv', -0.006466633429256527), ('311_category_School_Maintenance_added_zeros.csv', -0.0061099833776611665), ('cyclist_killed_sum.csv', -0.005328306774035426), ('311_category_Smoking_added_zeros.csv', -0.005150888193979814), ('311_category_Boilers_added_zeros.csv', -0

### What if the model used by the user were a linear regression instead of a random forest?

In [478]:
from sklearn.linear_model import LinearRegression

names_improvements = []
linreg = LinearRegression()
for name in candidate_names:
        candidate_dataset = pd.read_csv(os.path.join('nyc_indicators/', name),
                                        sep=SEPARATOR)
        numerical_column = [i for i in candidate_dataset.columns if i != 'time'][0]
        candidate_dataset = candidate_dataset.rename(columns={numerical_column: name.split('.')[0]})
        initial, final, improvement = compute_model_performance_improvement(crash_many_predictors.set_index('time'), 
                                                                            candidate_dataset.set_index('time'), 
                                                                            'crash_count', 
                                                                            'time', 
                                                                            adjusted_r2_score=True, 
                                                                            model=linreg)
        gains_in_r2_score.append(improvement)
        names_improvements.append((name, improvement))
        if improvement > 0:
            true_labels.append('gain')
        else:
            true_labels.append('loss')
print(sorted(names_improvements, key=lambda x: x[1]))

[('311_category_Illegal_parking_added_zeros.csv', -0.03130540327588831), ('311_category_Agency_Issues_added_zeros.csv', -0.03073419118268377), ('311_category_Benefit_Card_Replacement_added_zeros.csv', -0.03046877116163275), ('311_category_derelict_added_zeros.csv', -0.02452031173579836), ('311_category_unsanitary_added_zeros.csv', -0.022454693474382153), ('311_category_Food_Establishment_added_zeros.csv', -0.015877620756254485), ('311_category_Non-Emergency_Police_Matter_added_zeros.csv', -0.01332642089887011), ('311_category_Air_Quality_added_zeros.csv', -0.012813742017743296), ('311_category_Smoking_added_zeros.csv', -0.010698977550442764), ('311_category_Blocked_Driveway_added_zeros.csv', -0.010537410309341412), ('turnstile_count.csv', -0.010501254477746812), ('311_category_taxi_added_zeros.csv', -0.007892035639892766), ('311_category_Construction_added_zeros.csv', -0.007677094577480214), ('311_category_Litter_basket_added_zeros.csv', -0.007642806104633979), ('311_category_rodent_ad

### Ok. So now 43 out of 76 candidates are reducing the performance. It's not exactly a 'needle in a haystack' scenario when over 40% of the candidates bring gain, but let's see how our model works here. 

### First of all, we're gonna have to re-train our model to use ADJUSTED R2-SCORES. Note that the instances that compose the openml_training dataset use random forest, not linear regression! If it works well, it works as evidence that our model can adapt well.

In [479]:
def derive_adj_r2_score_from_dataset(dataset):
    '''
    Given a dataset with instances that can be used to train our model 
    (or test instances for said model), this function derives adjusted 
    r2 scores and their corresponding gains for each instance.
    '''
    adjusted_r2_scores_before = []
    adjusted_r2_scores_after = []
    gains_in_adjusted_r2_scores = []
    for index, row in dataset.iterrows():
        number_of_regressors_before = row['query_num_of_columns']
        r2_score_before = row['r2_score_before']
        
        number_of_regressors_after = row['query_num_of_columns'] + row['candidate_num_of_columns']
        r2_score_after = row['r2_score_after']

        number_of_samples = row['query_num_of_rows'] #after augmentation, this is also the number of samples
        
        adj_r2_score_before = compute_adjusted_r2_score(number_of_regressors_before,
                                                        r2_score_before, 
                                                        number_of_samples)
        
        adj_r2_score_after = compute_adjusted_r2_score(number_of_regressors_after, 
                                                       r2_score_after, 
                                                       number_of_samples)
        
        gain_in_adj_r2_score = (adj_r2_score_after - adj_r2_score_before)/np.fabs(adj_r2_score_before)
        
        adjusted_r2_scores_before.append(adj_r2_score_before)
        adjusted_r2_scores_after.append(adj_r2_score_after)
        gains_in_adjusted_r2_scores.append(gain_in_adj_r2_score)
        
    return adjusted_r2_scores_before, adjusted_r2_scores_after, gains_in_adjusted_r2_scores    

adj_r2_scores_before, adj_r2_scores_after, gains_in_adj_r2_scores = derive_adj_r2_score_from_dataset(openml_training)    

**** ERROR 62.0 61.0
**** ERROR 62.0 61.0
**** ERROR 62.0 61.0
**** ERROR 62.0 61.0


In [480]:
openml_training['adj_r2_score_before'] = adj_r2_scores_before
openml_training['adj_r2_scores_after'] = adj_r2_scores_after
openml_training['gain_in_adj_r2_score'] = gains_in_adj_r2_scores

In [481]:
openml_training['adj_class_pos_neg'] = ['gain' if row['gain_in_adj_r2_score'] > 0 else 'loss' 
                                        for index, row in openml_training.iterrows()]
openml_training_high_containment = openml_training.loc[openml_training['containment_fraction'] >= THETA]
feature_scaler, model = train_rbf_svm(openml_training_high_containment[FEATURES], 
                                      openml_training_high_containment['adj_class_pos_neg'])

### Let's remember what the current query/initial dataset is and what the candidates are:

In [482]:
crash_many_predictors.head()

,time,taxi_count,crash_count,taxispeed,streetlight_complaints,persons_injured,motorist_injured,pedestrians_injured
0,2012-07-01,354746,443,17.902601,37,143,104,30
1,2012-07-02,412337,475,16.507096,410,138,91,24
2,2012-07-03,495375,577,15.954152,300,187,139,34
3,2012-07-04,345717,353,17.808810,31,125,100,15
4,2012-07-05,417036,517,16.603352,290,124,86,30


### The goal is to see what labels are predicted now with this 'adjusted' model, and finally check how similar they are to the true labels. Note that this is an experiment on EFFECTIVENESS.

In [484]:
feature_vectors = []

for name in candidate_names:
    candidate_dataset = pd.read_csv(os.path.join('nyc_indicators/', name), 
                                    sep=SEPARATOR)
    numerical_column = [i for i in candidate_dataset.columns if i != 'time'][0]
    candidate_dataset = candidate_dataset.rename(columns={numerical_column: name.split('.')[0]})
    features = compute_features(crash_many_predictors.set_index('time'), 
                                candidate_dataset.set_index('time'), 
                                'time', 
                                'crash_count')
    feature_vectors.append(features[:-1])
    
predictions = model.predict(normalize_features(np.array(feature_vectors)))    

In [485]:
print(predictions, len(predictions))

['gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain'
 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain'
 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain'
 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain'
 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain'
 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain'
 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain' 'gain'
 'gain' 'gain' 'gain' 'gain' 'gain' 'gain'] 76


In [486]:
true_labels = []
linreg = LinearRegression()
for name in candidate_names:
    candidate_dataset = pd.read_csv(os.path.join('nyc_indicators/', name), 
                                    sep=SEPARATOR)
    numerical_column = [i for i in candidate_dataset.columns if i != 'time'][0]
    candidate_dataset = candidate_dataset.rename(columns={numerical_column: name.split('.')[0]})
    initial, final, improvement = compute_model_performance_improvement(crash_many_predictors.set_index('time'), 
                                                                        candidate_dataset.set_index('time'), 
                                                                        'crash_count', 
                                                                        'time', 
                                                                        adjusted_r2_score=True, 
                                                                        model=linreg)
    if improvement > 0:
        true_labels.append('gain')
    else:
        true_labels.append('loss')
print(np.array(true_labels), len(true_labels))

['loss' 'gain' 'loss' 'gain' 'loss' 'loss' 'gain' 'gain' 'loss' 'gain'
 'loss' 'loss' 'loss' 'loss' 'loss' 'loss' 'loss' 'loss' 'loss' 'loss'
 'gain' 'gain' 'loss' 'gain' 'gain' 'loss' 'loss' 'gain' 'gain' 'gain'
 'gain' 'loss' 'loss' 'loss' 'loss' 'loss' 'loss' 'gain' 'loss' 'gain'
 'gain' 'loss' 'gain' 'gain' 'gain' 'gain' 'gain' 'loss' 'gain' 'loss'
 'gain' 'gain' 'loss' 'loss' 'loss' 'gain' 'loss' 'loss' 'loss' 'gain'
 'gain' 'loss' 'loss' 'loss' 'loss' 'gain' 'gain' 'gain' 'loss' 'gain'
 'gain' 'gain' 'loss' 'loss' 'loss' 'loss'] 76


### Ok, so our method is not pruning anything for this particular example. Let's go back to r2 scores...

In [487]:
feature_scaler, model = train_rbf_svm(openml_training_high_containment[FEATURES], 
                                      openml_training_high_containment['class_pos_neg'])
predictions = model.predict(normalize_features(np.array(feature_vectors))) 

In [488]:
true_labels = []
linreg = LinearRegression()
for name in candidate_names:
    candidate_dataset = pd.read_csv(os.path.join('nyc_indicators/', name), 
                                    sep=SEPARATOR)
    numerical_column = [i for i in candidate_dataset.columns if i != 'time'][0]
    candidate_dataset = candidate_dataset.rename(columns={numerical_column: name.split('.')[0]})
    initial, final, improvement = compute_model_performance_improvement(crash_many_predictors.set_index('time'), 
                                                                        candidate_dataset.set_index('time'), 
                                                                        'crash_count', 
                                                                        'time', 
                                                                        adjusted_r2_score=False, 
                                                                        model=linreg)
    if improvement > 0:
        true_labels.append('gain')
    else:
        true_labels.append('loss')

print(classification_report(true_labels, predictions))

              precision    recall  f1-score   support

        gain       0.60      0.91      0.72        43
        loss       0.64      0.21      0.32        33

    accuracy                           0.61        76
   macro avg       0.62      0.56      0.52        76
weighted avg       0.62      0.61      0.55        76



### Ok. Note that, in this case, the minority corresponds to 'loss' -- not to 'gain', what would be ideal. But that's ok, we can fix that later. 

### Note that the effectiveness is pretty ok for class 'gain' but could be better for class 'loss': the low recall for the latter is to blame... 

### Before we try to understand EFFICIENCY, and see what kind of gains we would have if we used our model before using ARDA, let's see how a containment baseline would work in this case.

In [490]:
feature_vectors = []
containment_ratios = []
for name in candidate_names:
    candidate_dataset = pd.read_csv(os.path.join('nyc_indicators/', name), 
                                    sep=SEPARATOR)
    numerical_column = [i for i in candidate_dataset.columns if i != 'time'][0]
    candidate_dataset = candidate_dataset.rename(columns={numerical_column: name.split('.')[0]})
    output = compute_features(crash_many_predictors.set_index('time'), 
                              candidate_dataset.set_index('time'), 
                              'time', 
                              'crash_count')
    feature_vectors.append(output[:-1])
    containment_ratios.append((name, output[-1]))

### Let's see the containment ratios in order to figure out some kind of threshold for a baseline.

In [491]:
print(containment_ratios)

[('311_category_Agency_Issues_added_zeros.csv', 1.0), ('311_category_SCRIE_added_zeros.csv', 1.0), ('cyclist_killed_sum.csv', 1.0), ('311_category_electric_added_zeros.csv', 1.0), ('311_category_Illegal_parking_added_zeros.csv', 1.0), ('311_category_Vacant_Lot_added_zeros.csv', 1.0), ('311_category_consumer_complaint_added_zeros.csv', 1.0), ('weather_temperature_mean.csv', 1.0), ('311_category_Litter_basket_added_zeros.csv', 1.0), ('311_category_dof_added_zeros.csv', 1.0), ('pedestrians_killed_sum.csv', 1.0), ('311_category_Construction_added_zeros.csv', 0.9397672826830937), ('311_category_DOH_New_License_Application_Request_added_zeros.csv', 1.0), ('311_category_Sidewalk_Condition_added_zeros.csv', 1.0), ('persons_killed_sum.csv', 1.0), ('311_category_Food_Establishment_added_zeros.csv', 1.0), ('311_category_Drinking_added_zeros.csv', 1.0), ('311_category_unsanitary_added_zeros.csv', 1.0), ('311_category_rodent_added_zeros.csv', 1.0), ('turnstile_count.csv', 1.0), ('311_category_floor

### Ok, so most of them are basically equal to one. Let's do the following baseline: if a containment ratio is below 
### 0.6, let's assume the predicted class will be 'loss'; otherwise, it will be gain. Let's see how this compares to 
### our model.

In [492]:
containment_based_predictions = ['loss' if elem[1] < 0.6 else 'gain' for elem in containment_ratios]
print(classification_report(true_labels, containment_based_predictions))

              precision    recall  f1-score   support

        gain       0.56      0.95      0.71        43
        loss       0.33      0.03      0.06        33

    accuracy                           0.55        76
   macro avg       0.45      0.49      0.38        76
weighted avg       0.46      0.55      0.42        76



### The results were significantly worse. This is beneficial for us. Now let's implement RIFS, the feature selection algorithm in ARDA, and start digging into efficiency.

In [493]:
from sparsereg.model.base import STRidge # the stype of sparsereg they use is not clear in the ARDA paper
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

RANDOM_PREFIX = 'random_feature_'

def augment_with_random_features(dataset, number_of_random_features):
    '''
    Given a dataset and a number of random features, this function derives
    random features that are based on the original ones in the dataset
    '''
    mean = dataset.T.mean()
   # print(dataset.T)
    cov = dataset.T.cov()
    features = np.random.multivariate_normal(mean, cov, number_of_random_features)
    for i in range(number_of_random_features):
        dataset[RANDOM_PREFIX + str(i)] = features[i,:]
    return dataset

def combine_rankings(rf_coefs, regression_coefs, feature_names, lin_comb_coef=0.5):
    '''
    Given feature coefficients computed with different methods (random forest and a regularized 
    regression), we scale their values, combine them linearly, rescale them and rank their  
    corresponding features according to their values
    '''
    
    # random forest coefficients are already in interval [0, 1]; let's pre-scale the regression 
    # coefficients, which are all over the place
    min_value = min(regression_coefs)
    shifted = np.array([elem + min_value*-1 for elem in regression_coefs])
    normalized_regression_coefs = shifted/sum(shifted)
    
    combined_scores = lin_comb_coef*rf_coefs + (1-lin_comb_coef)*normalized_regression_coefs
    ranked_features = sorted([(feat_name, score) for feat_name, score in zip(feature_names, combined_scores)], 
                             key=lambda x: x[1], 
                             reverse=True)
    return [elem[0] for elem in ranked_features]

def feature_in_front_of_random(feature_name, ranking):
    '''
    This function checks whether there are random features before 
    feature_name
    '''
    random_not_seen = True
    for feat in ranking:
        if RANDOM_PREFIX in feat:
            return 0
        if feat == feature_name:
            return 1
    return 0

def aggregate_features_by_quality(rankings):
    '''
    Given feature rankings, this function counts, for every non-random feature, 
    how many times it occured before every random feature
    '''
    feature_names = [elem for elem in rankings[0] if RANDOM_PREFIX not in elem]
    
    feats_quality = {}
    for feature in feature_names:
        for rank in rankings:
            if feature in feats_quality:
                feats_quality[feature] += feature_in_front_of_random(feature, rank)
            else:
                feats_quality[feature] = feature_in_front_of_random(feature, rank)
    sorted_feats =  sorted(feats_quality.items(), 
                           key=lambda x: x[1], 
                           reverse=True)
    return [(elem[0], elem[1]/len(rankings)) for elem in sorted_feats]
    
def random_injection_feature_selection(augmented_dataset_features, 
                                       target_column_data, 
                                       tau, 
                                       eta, 
                                       k_random_seeds):
    '''
    This is ARDA's feature selection algorithm RIFS. Given an augmented dataset, ideally 
    created through joins over sketches over the original datasets, a threshold tau for 
    quality of features, a fraction eta of random features to inject, and k random seeds to perform 
    k experiments in a reproducible way, it selects the features that should be used in the augmented 
    dataset
    '''
    number_of_random_features = int(np.ceil(eta*augmented_dataset_features.shape[1]))
    augmented_dataset_with_random = augment_with_random_features(augmented_dataset_features, 
                                                                 number_of_random_features)
    
    # Now we obtain rankings using random forests and sparse regression models
    ## the paper does not say what hyperparameters were used in the experiment
    rankings = []
    for seed in range(k_random_seeds):
        rf = RandomForestRegressor(n_estimators=100, random_state=seed)
        rf.fit(augmented_dataset_features, target_column_data)
        rf_coefs = rf.feature_importances_
        
        ## coef = STRidge().fit(augmented_dataset_features, target_column_data).coef_
        ## print(coef, augmented_dataset_features.columns)
        ## print(len(rf.feature_importances_))
        
        ## This version of lasso is giving lower weights to random features, which is good
        lasso = Lasso(random_state=seed)
        lasso.fit(augmented_dataset_features, target_column_data)
        lasso_coefs = lasso.coef_
        rank = combine_rankings(rf_coefs, lasso_coefs, augmented_dataset_features.columns)
        rankings.append(rank)
    
    # Now, for each non-random feature, we get the number of times it appeared in front of 
    ## all random features
    sorted_features = aggregate_features_by_quality(rankings)
    return [elem[0] for elem in sorted_features if elem[1] >= tau]
    
def wrapper_algorithm(augmented_dataset, target_name, key, thresholds_T, eta, k_random_seeds):
    '''
    This function searches for the best subset of features by doing an exponential search
    '''
    X_train, X_test, y_train, y_test = train_test_split(augmented_dataset.drop(target_name, axis=1), 
                                                        augmented_dataset[target_name], 
                                                        test_size=0.33,
                                                        random_state=42)
    current_r2_score = float('-inf')
    linreg = LinearRegression()
    selected = []
    for tau in thresholds_T:
        selected = random_injection_feature_selection(augmented_dataset.drop([target_name], axis=1), 
                                                      augmented_dataset[target_name],
                                                      tau, 
                                                      eta, 
                                                      k_random_seeds)
        linreg.fit(X_train[selected], y_train)
        y_pred = linreg.predict(X_test[selected])
        new_r2_score = r2_score(y_test, y_pred)
        if  new_r2_score > current_r2_score:
            current_r2_score = new_r2_score
        else:
            break
    return selected


### Ok, so now we have the code for RIFS. What we need to do now is check how it behaves when we use *all* available features, and see how things compare in time.

In [574]:
import time

def check_efficiency_with_ida(base_dataset, 
                              dataset_directory, 
                              key, 
                              target_name, 
                              training_data, 
                              tau, 
                              eta, 
                              k_random_seeds, 
                              mean_data_imputation=True, 
                              rename_numerical=True, 
                              separator='|'):
    '''
    This function compares the time to run RIFS with and without pre-pruning with IDA
    '''
    
    #Step 1: do the join with every candidate dataset in dataset_directory. 
    ## This has to be done both with and without IDA. 
    augmented_dataset = join_datasets(base_dataset, dataset_directory, key, rename_numerical=rename_numerical, separator=separator)
    augmented_dataset = augmented_dataset.loc[:,~augmented_dataset.columns.duplicated()] #removing duplicate columns
    print('Done creating the augmented dataset')
    
    #Step 2: let's see how much time it takes to select features with RIFS, injecting 20% of random features
    time1 = time.time()
    selected_all = wrapper_algorithm(augmented_dataset, target_name, key, [0.2, 0.4, 0.6, 0.8], 0.2, 10)
    time2 = time.time()
    print('time to run RIFS', (time2-time1)*1000.0, 'ms')
    
    #Step 3: let's train our IDA model over the training dataset
    time1 = time.time()
    feature_scaler, model = train_rbf_svm(training_data[FEATURES], 
                                          training_data['class_pos_neg'])
    time2 = time.time()
    print('time to train our model', (time2-time1)*1000.0, 'ms')
    
    #Step 4: generate a label for every feature in the augmented dataset
    time1 = time.time()
    candidate_names = set(augmented_dataset.columns) - set(base_dataset.columns)
    feature_vectors = []
    for name in candidate_names:
        candidate_dataset = augmented_dataset.reset_index()[[key, name]]
        #print('getting features for candidate column', name)
        #print(candidate_dataset.set_index(key))
        features = compute_features(base_dataset.set_index(key), 
                                    candidate_dataset.set_index(key), 
                                    key, 
                                    target_name, 
                                    augmented_dataset=augmented_dataset)
        feature_vectors.append(features[:-1])
    predictions = model.predict(normalize_features(np.array(feature_vectors))) 

    candidates_to_keep = [name for name, pred in zip(candidate_names, predictions) if pred == 'gain']
    time2 = time.time()
    print('time to predict what candidates to keep', (time2-time1)*1000.0, 'ms')
    
    #Step 5: run RIFS only considering the features to keep   
    pruned = augmented_dataset[base_dataset.set_index(key).columns.to_list() + candidates_to_keep]
    
    time1 = time.time()
    selected_pruned = wrapper_algorithm(pruned, 
                                        target_name, 
                                        key, 
                                        [0.2, 0.4, 0.6, 0.8], 
                                        0.2, 
                                        10)
    time2 = time.time()
    print('time to run RIFS over features to keep', (time2-time1)*1000.0, 'ms')
    print('size of entire dataset', augmented_dataset.shape[1], 'size of pruned', pruned.shape[1])
    return selected_all, candidates_to_keep, selected_pruned

#selected_all, candidates_to_keep, selected_pruned = check_efficiency_with_ida(crash_many_predictors, 
#                                                                            'nyc_indicators/', 
#                                                                            'time', 
#                                                                            'crash_count', 
#                                                                            openml_training_high_containment, 
#                                                                            0.7, 
#                                                                            0.2,
#                                                                            [42, 17, 23, 2, 5, 19, 37, 41, 13, 33])

In [500]:
from sklearn.metrics import r2_score
augmented_dataset = join_datasets(crash_many_predictors, 'nyc_indicators/', 'time')

X_train, X_test, y_train, y_test = train_test_split(augmented_dataset.drop(['crash_count'], axis=1), 
                                                    augmented_dataset['crash_count'], 
                                                    test_size=0.33, 
                                                    random_state=42)
linreg = LinearRegression()
linreg.fit(X_train[selected_all], y_train)
y_pred = linreg.predict(X_test[selected_all])
r2_score_initial = r2_score(y_test.to_list(), y_pred)
#print('r2_score for select_all (no pruning)', r2_score_initial)

linreg.fit(X_train[selected_pruned], y_train)
y_pred = linreg.predict(X_test[selected_pruned])
r2_score_initial = r2_score(y_test.to_list(), y_pred)
#print('r2_score for select_pruned (pruning)', r2_score_initial)


### Now let's see how the IDA pruning works for a use case involving college columns

In [503]:
initial_college_dataset = pd.read_csv('datasets_for_use_cases/companion-datasets/college-debt-v2.csv')
initial_college_dataset = initial_college_dataset.fillna(initial_college_dataset.mean())

In [504]:
initial_college_dataset.head()

,UNITID,PCTFLOAN,PCIP16,PPTUG_EF,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,SATMTMID,SATVRMID,SATWRMID,UGDS,DEBT_EARNINGS_RATIO
0,12268508,0.550002,0.003175,0.232991,0.505208,0.187607,0.164215,0.03452,528.1225,521.880734,515.276297,3141.540889,49
1,207564,0.475000,0.000000,0.229700,0.295300,0.029100,0.064700,0.00510,528.1225,521.880734,515.276297,2164.000000,36
2,420024,0.812500,0.000000,0.231500,0.280800,0.566500,0.049300,0.00000,528.1225,521.880734,515.276297,203.000000,127
3,164492,0.746500,0.000000,0.262100,0.651800,0.125800,0.102200,0.01230,528.1225,521.880734,515.276297,1057.000000,76
4,234085,0.458900,0.032100,0.000000,0.799200,0.060700,0.058400,0.04200,575.0000,575.000000,515.276297,1713.000000,53


In [575]:
selected_all, candidates_to_keep, selected_pruned = check_efficiency_with_ida(initial_college_dataset, 
                                                                              'datasets_for_use_cases/companion-datasets/college-debt-single-column/', 
                                                                              'UNITID', 
                                                                              'DEBT_EARNINGS_RATIO', 
                                                                              openml_training_high_containment, 
                                                                              0.7, 
                                                                              0.2, 
                                                                              [42, 17, 23, 2, 5, 19, 37, 41, 13, 33], 
                                                                              rename_numerical=False, 
                                                                              separator=',')

Done creating the augmented dataset
time to run RIFS 1821234.0319156647 ms
time to train our model 653.2838344573975 ms
time to predict what candidates to keep 163886.07907295227 ms
time to run RIFS over features to keep 537127.8910636902 ms
size of entire dataset 768 size of pruned 299


In [579]:
print(len(selected_all), len(selected_pruned))
print(sorted(selected_all))
print(sorted(selected_pruned))

35 20
['CONTROL_y', 'HBCU_y', 'NPT41_PRIV_x', 'NPT4_PRIV_x', 'OPEID6_y', 'OPEID_x', 'OPEID_y', 'PCIP11_x', 'PCIP12_x', 'PCIP13_x', 'PCIP49_x', 'PCIP52_x', 'PCTFLOAN', 'PCTFLOAN_r_x', 'PCTPELL_x', 'PCTPELL_y', 'PPTUG_EF', 'PPTUG_EF_r_y', 'PREDDEG_y', 'RET_FT4_x', 'RET_FTL4_x', 'SAT_AVG_ALL_x', 'UG25ABV_x', 'UGDS', 'UGDS_ASIAN', 'UGDS_ASIAN_r_y', 'UGDS_BLACK', 'UGDS_BLACK_r_x', 'UGDS_HISP', 'UGDS_HISP_r_y', 'UGDS_NRA_x', 'UGDS_UNKN_x', 'UGDS_WHITE', 'UGDS_WHITE_r_x', 'UGDS_r_y']
['CONTROL_y', 'HBCU_y', 'OPEID6_y', 'OPEID_x', 'OPEID_y', 'PCIP47_x', 'PCIP52_x', 'PCTFLOAN', 'PCTFLOAN_r_x', 'PCTPELL_x', 'PCTPELL_y', 'PPTUG_EF', 'PREDDEG_y', 'UGDS', 'UGDS_ASIAN', 'UGDS_BLACK', 'UGDS_BLACK_r_x', 'UGDS_HISP', 'UGDS_HISP_r_y', 'UGDS_WHITE']


### The pruning was far more substantial this time. Let's see the quality of the results

In [580]:
augmented_dataset = join_datasets(initial_college_dataset, 
                                  'datasets_for_use_cases/companion-datasets/college-debt-single-column/', 
                                  'UNITID',
                                  rename_numerical=False,
                                  separator=',')
augmented_dataset = augmented_dataset.loc[:,~augmented_dataset.columns.duplicated()] #removing duplicate columns

X_train, X_test, y_train, y_test = train_test_split(augmented_dataset.drop(['DEBT_EARNINGS_RATIO'], axis=1), 
                                                    augmented_dataset['DEBT_EARNINGS_RATIO'], 
                                                    test_size=0.33, 
                                                    random_state=42)

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train[selected_all], y_train)
y_pred = rf.predict(X_test[selected_all])
r2_score_initial = r2_score(y_test.to_list(), y_pred)
print('r2_score for select_all (no pruning)', r2_score_initial)

rf.fit(X_train[selected_pruned], y_train)
y_pred = rf.predict(X_test[selected_pruned])
r2_score_initial = r2_score(y_test.to_list(), y_pred)
print('r2_score for select_pruned (pruning)', r2_score_initial)

r2_score for select_all (no pruning) 0.701455005905175
r2_score for select_pruned (pruning) 0.6696787586343466


#### Ok. So in this case the quality did drop a bit. Now let's see the concrete quality of this classifier considering the labels for each candidate.

In [587]:
def assess_classifier_quality(classifier, 
                              base_dataset, 
                              dataset_directory, 
                              key, 
                              target_name, 
                              rename_numerical=True, 
                              separator='|'):
    '''
    This function generates true labels and predictions for a set of candidate datasets and 
    assesses the quality of the classifier
    '''
    
    augmented_dataset = join_datasets(base_dataset, 
                                      dataset_directory, 
                                      key, 
                                      rename_numerical=rename_numerical, 
                                      separator=separator)
    augmented_dataset = augmented_dataset.loc[:,~augmented_dataset.columns.duplicated()]
    
    candidate_names = set(augmented_dataset.columns) - set(base_dataset.columns)
    feature_vectors = []
    labels = []
    for name in candidate_names:
        candidate_dataset = augmented_dataset.reset_index()[[key, name]]
        
        features = compute_features(base_dataset.set_index(key), 
                                    candidate_dataset.set_index(key), 
                                    key, 
                                    target_name, 
                                    augmented_dataset=augmented_dataset)
        feature_vectors.append(features[:-1])
        
        initial, final, improvement = compute_model_performance_improvement(base_dataset.set_index(key), 
                                                                            candidate_dataset.set_index(key), 
                                                                            target_name, 
                                                                            key, 
                                                                            adjusted_r2_score=False)
        if improvement > 0: 
            labels.append('gain')
        else:
            labels.append('loss')
    predictions = classifier.predict(normalize_features(np.array(feature_vectors))) 
    print(classification_report(labels, predictions))

In [588]:
feature_scaler, model = train_rbf_svm(openml_training_high_containment[FEATURES], 
                                      openml_training_high_containment['class_pos_neg'])

assess_classifier_quality(model, 
                          initial_college_dataset, 
                          'datasets_for_use_cases/companion-datasets/college-debt-single-column/', 
                          'UNITID', 
                          'DEBT_EARNINGS_RATIO', 
                              rename_numerical=False, 
                              separator=',')

              precision    recall  f1-score   support

        gain       0.23      0.44      0.31       152
        loss       0.82      0.64      0.72       603

    accuracy                           0.60       755
   macro avg       0.53      0.54      0.51       755
weighted avg       0.70      0.60      0.63       755

